# Conceptual and Statistical Introduction

## The statistical problem of high dimensionality

In high-dimensional biological data, many features are correlated. Variance measures how spread out data points are around their mean. Covariance measures how features vary together.

## Geometric intuition

PCA rotates the coordinate system to align with directions of maximum variance. These directions correspond to eigenvectors of the covariance matrix and represent the most informative axes of variation.

## Drug development relevance

Omics datasets often contain thousands of variables but few samples. PCA helps separate signal from noise and identify dominant biological processes before downstream biomarker analysis.

## Intuitive Understanding of PCA (Feynman-Style)

Before applying PCA to real biological data, we build intuition using a very small, concrete example.

**PCA asks a simple question:**

> In which direction are the data points most spread out?

That direction captures the most information and becomes the **first principal component**.

### A Simple Numeric Example

Consider the following four points:

- (1, 2)
- (2, 4)
- (3, 6)
- (4, 8)

As x increases by 1, y increases by 2. The points lie almost perfectly along a straight line.

Intuitively, almost all information lies along **one direction**, not two.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# Define a small 2D dataset with strong correlation
X = np.array([
    [1, 2],
    [2, 4],
    [3, 6],
    [4, 8]
])

# Plot the original data points
plt.figure(figsize=(6, 6))
plt.scatter(X[:, 0], X[:, 1])
plt.xlabel('Feature 1')
plt.ylabel('Feature 2')
plt.title('Original Data Points')
plt.grid(True)
plt.show()

### Why Mean-Centering Is Necessary

PCA measures variance **around the mean**.

If the data cloud is not centered at the origin, distances are measured incorrectly.

Mean-centering shifts the cloud so its center lies at (0, 0), without changing its shape.

In [ ]:
# Compute the mean of each feature
mean = np.mean(X, axis=0)

# Center the data
X_centered = X - mean

# Plot mean-centered data
plt.figure(figsize=(6, 6))
plt.scatter(X_centered[:, 0], X_centered[:, 1])
plt.axhline(0, color='gray', linestyle='--')
plt.axvline(0, color='gray', linestyle='--')
plt.xlabel('Centered Feature 1')
plt.ylabel('Centered Feature 2')
plt.title('Mean-Centered Data')
plt.grid(True)
plt.show()

### Direction of Maximum Variance

The direction along which the projected points are most spread out captures the maximum variance.

In this example, that direction aligns closely with the line y = 2x.

PCA mathematically finds this direction by maximizing variance.

# PCA for Biomarker Discovery (Drug Development)

## 1. Motivation

High-dimensional biological data contains correlated features. PCA reduces dimensionality while preserving dominant variance patterns.

## 2. Generating Synthetic Biomarker Data

Rows represent samples and columns represent genes or proteins.

In [ ]:
import pandas as pd

np.random.seed(42)

num_samples = 50
num_features = 100

data = np.random.rand(num_samples, num_features) * 10

df = pd.DataFrame(data, columns=[f'Gene_{i+1}' for i in range(num_features)])

df.head()

## 3. Standardizing the Data

PCA is sensitive to feature scale. Standardization ensures equal contribution from all features.

In [ ]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
scaled_data = scaler.fit_transform(df)

## 4. Performing PCA

Principal components capture orthogonal directions of maximum variance.

In [ ]:
from sklearn.decomposition import PCA

pca = PCA(n_components=10)
principal_components = pca.fit_transform(scaled_data)

pc_df = pd.DataFrame(principal_components, columns=[f'PC_{i+1}' for i in range(10)])

pc_df.head()

## 5. Explained Variance Analysis

Explained variance indicates how much information each component retains.

In [ ]:
explained_variance_ratio = pca.explained_variance_ratio_
cumulative_explained_variance = np.cumsum(explained_variance_ratio)

plt.figure(figsize=(10, 6))
plt.plot(range(1, len(explained_variance_ratio)+1), explained_variance_ratio, 'o-', label='Individual')
plt.plot(range(1, len(explained_variance_ratio)+1), cumulative_explained_variance, 'o-', label='Cumulative')
plt.xlabel('Principal Component')
plt.ylabel('Explained Variance')
plt.legend()
plt.show()